## Importing libraries

In [1]:
# import sqlalchemy
from google.cloud import bigquery
from sqlalchemy.engine import create_engine
from google.oauth2 import service_account

import pickle
import numpy as np
import pandas as pd
import re
import geopy.distance

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## Data reading 

In [2]:
credentials_json_path = "credentials_view.json"

credentials = service_account.Credentials.from_service_account_file(
    credentials_json_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bqclient = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [3]:
# PROJECT_ID = credentials.project_id
# DB_NAME = "house_data"
# DATASET = "london_house_prices"

In [4]:
# Download a table.
table = bigquery.TableReference.from_string(
    "candidate-testing.house_data.london_house_prices"
)
rows = bqclient.list_rows(
    table
)
df = rows.to_dataframe()

In [5]:
df.head()

,address,type,bedrooms,latitude,longitude,area,price,tenure,is_newbuild,date
0,"Flat 3, Warwick Apartments, 132, Cable Street,...",Flat,1,51.51092,-0.06250,E1,299995,Leasehold,1,2014-01-28 00:00:00+00:00
1,"Flat 3, Warwick Apartments, 132, Cable Street,...",Flat,1,51.51092,-0.06250,E1,445000,Leasehold,0,2019-06-20 00:00:00+00:00
2,"25, Varden Street, London, Greater London E1 2AW",Terraced,3,51.51576,-0.06128,E1,300000,Freehold,0,2007-04-20 00:00:00+00:00
3,"25, Varden Street, London, Greater London E1 2AW",Terraced,3,51.51576,-0.06128,E1,1350000,Freehold,0,2020-11-27 00:00:00+00:00
4,"22b Petticoat Tower, Petticoat Square, London,...",Flat,1,51.51575,-0.07714,E1,249950,Leasehold,0,2014-11-07 00:00:00+00:00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690063 entries, 0 to 690062
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   address      690063 non-null  object             
 1   type         690063 non-null  object             
 2   bedrooms     690063 non-null  int64              
 3   latitude     690063 non-null  float64            
 4   longitude    690063 non-null  float64            
 5   area         690063 non-null  object             
 6   price        690063 non-null  int64              
 7   tenure       690063 non-null  object             
 8   is_newbuild  690063 non-null  int64              
 9   date         690063 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(4)
memory usage: 52.6+ MB


In [7]:
# df['address'].value_counts()

## Data cleaning and preprocessing

In [8]:
df = df.drop_duplicates().reset_index(drop=True)
df.shape

(241007, 10)

In [9]:
df.columns

Index(['address', 'type', 'bedrooms', 'latitude', 'longitude', 'area', 'price',
       'tenure', 'is_newbuild', 'date'],
      dtype='object')

In [10]:
df = df[~((df.latitude==0) & ((df.longitude==0)))]
df.shape

(240911, 10)

In [11]:
#if everything is same except type we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='type'], keep=False)

#if everything is same except bedrooms we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='bedrooms'], keep=False)

#if everything is same except latitude and longitude we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='latitude' and col!='longitude'])

#if everything is same except area we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='area'], keep=False)

#if everything is same except price we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='price'], keep=False)

#if everything is same except tenure we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='tenure'], keep=False)

#if everything is same except is_newbuild we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='is_newbuild'], keep=False)

In [12]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [13]:
df.shape

(238641, 10)

In [14]:
df = remove_outlier(df, 'price')
df.shape

(220966, 10)

## Feature engineering

### Address column

In [15]:
df['address'] = df['address'].apply(lambda x: x.lower().replace("'",""))

In [16]:
sample_sentence_1 = '6, Polworth, london, Greater London SW16 2EU'.lower()
sample_sentence_2 = '6, Polworth Street, London, Greater London SW16 2EU'.lower()

In [17]:
def extract_road(x):
    """This function extracts the road/street/... from input string
    """
    road_matcher = re.compile(r'(\w+\s?\w+),\s?london,')
    res = road_matcher.findall(x)
    if res:
        return res[0]
    return None

print(extract_road(sample_sentence_1))
print(extract_road(sample_sentence_2))

polworth
polworth street


In [18]:
df['road'] = df['address'].apply(extract_road)

In [19]:
top5_roads = df['road'].value_counts().index[:5]

In [20]:
df['road'] = np.where(df['road'].isin(top5_roads), df['road'], 'Other')

In [21]:
# uniuqe_ll = df[['latitude','longitude']].drop_duplicates().values
# uniuqe_ll

In [23]:
df.head(2)

,address,type,bedrooms,latitude,longitude,area,price,tenure,is_newbuild,date,road
0,"flat 3, warwick apartments, 132, cable street,...",Flat,1,51.51092,-0.0625,E1,299995,Leasehold,1,2014-01-28 00:00:00+00:00,Other
1,"flat 3, warwick apartments, 132, cable street,...",Flat,1,51.51092,-0.0625,E1,445000,Leasehold,0,2019-06-20 00:00:00+00:00,Other


### Date column

In [24]:
df.date

0        2014-01-28 00:00:00+00:00
1        2019-06-20 00:00:00+00:00
2        2007-04-20 00:00:00+00:00
4        2014-11-07 00:00:00+00:00
5        2019-05-10 00:00:00+00:00
                    ...           
241002   2019-09-24 00:00:00+00:00
241003   2006-03-27 00:00:00+00:00
241004   2006-09-07 00:00:00+00:00
241005   2020-08-11 00:00:00+00:00
241006   2020-09-04 00:00:00+00:00
Name: date, Length: 220966, dtype: datetime64[ns, UTC]

In [25]:
max(df.date)

Timestamp('2021-03-26 00:00:00+0000', tz='UTC')

In [26]:
min(df.date)

Timestamp('1995-01-03 00:00:00+0000', tz='UTC')

In [27]:
day = pd.to_datetime(np.datetime64(min(df.date))).tz_localize(None)

C:\Users\SOLOLE~1\AppData\Local\Temp/ipykernel_23344/2231202990.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  day = pd.to_datetime(np.datetime64(min(df.date))).tz_localize(None)


In [28]:
df['date'] = df['date'].dt.tz_localize(None)

In [29]:
df['min_date'] = day

In [30]:
df['days_from_min_date'] = (df.date - df.min_date).dt.days

In [31]:
df.sample(4)

,address,type,bedrooms,latitude,longitude,area,price,tenure,is_newbuild,date,road,min_date,days_from_min_date
210291,"flat a, 13, ulverstone road, london, greater l...",Flat,3,51.43683,-0.10650,SE27,340000,Leasehold,0,2017-11-10,Other,1995-01-03,8347
237670,"24, hamilton road, london, greater london sw19...",Terraced,3,51.41704,-0.19174,SW19,780000,Freehold,0,2020-12-02,Other,1995-01-03,9465
42855,"123, sutton court, london, greater london w4 3ef",Flat,3,51.48674,-0.26964,W4,675000,Leasehold,0,2019-06-07,Other,1995-01-03,8921
189207,"86, queen mary road, upper norwood, london, gr...",Terraced,3,51.42158,-0.10353,SE19,40000,Freehold,0,1996-04-12,Other,1995-01-03,465


### Latitude Longitude to kilometers from center

In [32]:
def distance_from_center(coordinate):
    latitude = float(coordinate.split(',')[0])
    longitude = float(coordinate.split(',')[1])
    center = [51.509865, -0.118092]
    coordinate = [latitude, longitude]
    distance = geopy.distance.distance(center, coordinate).km
    return distance

In [33]:
df['coordinate'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)

In [34]:
df['distance_from_center'] = df.coordinate.apply(distance_from_center)

In [35]:
df.distance_from_center.max()

19.097762996435154

## Train test split and one_hot_encode

In [36]:
df.columns

Index(['address', 'type', 'bedrooms', 'latitude', 'longitude', 'area', 'price',
       'tenure', 'is_newbuild', 'date', 'road', 'min_date',
       'days_from_min_date', 'coordinate', 'distance_from_center'],
      dtype='object')

In [37]:
data = df.drop(['address','date', 'min_date','coordinate'],axis=1)

In [38]:
# X = data.drop(['price'], axis=1)
# y = data['price']

In [39]:
train, test = train_test_split(data, test_size=0.3, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)

In [40]:
train.shape, test.shape

((154676, 11), (33145, 11))

In [41]:
def one_hot_encode_fit_transform(data):
    enc = OneHotEncoder(handle_unknown = 'ignore')
    enc.fit(data[['type', 'area', 'tenure', 'road']])
    encodings = enc.transform(data[['type', 'area', 'tenure', 'road']]).toarray()
    cols = enc.get_feature_names()
    one_hot_data = pd.DataFrame(data=encodings, columns=cols, index=data.index)
    data = data.drop(['type', 'area', 'tenure', 'road'], axis=1)
    data = data.join(one_hot_data)
    with open("encoder", "wb") as f: 
        pickle.dump(enc, f)
    return data

In [42]:
def one_hot_encode_transform(data):
    with open("encoder", "rb") as f:
        enc = pickle.load(f)
    encodings = enc.transform(data[['type', 'area', 'tenure', 'road']]).toarray()
    cols = enc.get_feature_names()
    one_hot_data = pd.DataFrame(data=encodings, columns=cols, index=data.index)
    data = data.drop(['type', 'area', 'tenure', 'road'], axis=1)
    data = data.join(one_hot_data)
    return data

In [43]:
train = one_hot_encode_fit_transform(train)

In [44]:
test = one_hot_encode_transform(test)
val = one_hot_encode_transform(val)

In [45]:
train.head(2)

,bedrooms,latitude,longitude,price,is_newbuild,days_from_min_date,distance_from_center,x0_Detached,x0_Flat,x0_Semi-Detached,...,x1_W8,x1_W9,x2_Freehold,x2_Leasehold,x3_Other,x3_green road,x3_high road,x3_high street,x3_hill road,x3_park road
6259,3,51.61917,-0.00704,110000,0,1284,14.394216,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
105328,3,51.54970,-0.17552,305000,0,2304,5.960242,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [46]:
test.head(5)

,bedrooms,latitude,longitude,price,is_newbuild,days_from_min_date,distance_from_center,x0_Detached,x0_Flat,x0_Semi-Detached,...,x1_W8,x1_W9,x2_Freehold,x2_Leasehold,x3_Other,x3_green road,x3_high road,x3_high street,x3_hill road,x3_park road
240876,2,51.41046,-0.21414,620000,0,8954,12.917977,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
143471,4,51.46698,-0.14225,560000,0,3796,5.057738,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7966,3,51.55665,-0.05969,301000,0,9332,6.596760,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
66478,2,51.49957,0.02609,417950,1,9278,10.076349,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
41043,4,51.50583,-0.27742,137500,0,514,11.071022,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [47]:
bedrooms_mean = train.bedrooms.mean()
bedrooms_std = train.bedrooms.std()
latitude_mean = train.latitude.mean()
latitude_std = train.latitude.std()
longitude_mean = train.longitude.mean()
longitude_std = train.longitude.std()
days_from_min_date_mean = train.days_from_min_date.mean()
days_from_min_date_std = train.days_from_min_date.std()
distance_from_center_mean = train.distance_from_center.mean()
distance_from_center_std = train.distance_from_center.std()

In [48]:
print(bedrooms_mean,
      '=====',
      bedrooms_std,
      '=====',
      latitude_mean,
      '=====',
      latitude_std, 
      '=====',
      longitude_mean, 
      '=====',
      longitude_std, 
      '=====',
      days_from_min_date_mean, 
      '=====',
      days_from_min_date_std,
      '=====',
      distance_from_center_mean,
      '=====',
      distance_from_center_std)

2.6061767824355426 ===== 1.1694933172332571 ===== 51.51417806518142 ===== 0.06153182726811449 ===== -0.10760776920789263 ===== 0.09535970982651915 ===== 6157.339425638108 ===== 3006.2284278681286 ===== 8.967594045364942 ===== 3.3223895064504845


In [49]:
def normalize(data):
    data.bedrooms = (data.bedrooms - bedrooms_mean) / bedrooms_std
    data.latitude = (data.latitude - latitude_mean) / latitude_std
    data.longitude = (data.longitude - longitude_mean) / longitude_std
    data.days_from_min_date = (data.days_from_min_date - days_from_min_date_mean) / days_from_min_date_std
    data.distance_from_center = (data.distance_from_center - distance_from_center_mean) / distance_from_center_std
    return data

In [50]:
train = normalize(train)
test = normalize(test)
val = normalize(val)

In [51]:
train

,bedrooms,latitude,longitude,price,is_newbuild,days_from_min_date,distance_from_center,x0_Detached,x0_Flat,x0_Semi-Detached,...,x1_W8,x1_W9,x2_Freehold,x2_Leasehold,x3_Other,x3_green road,x3_high road,x3_high street,x3_hill road,x3_park road
6259,0.336747,1.706303,1.054615,110000,0,-1.621081,1.633349,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
105328,0.336747,0.577294,-0.712169,305000,0,-1.281785,-0.905177,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
99882,1.191818,1.493567,0.002074,389995,0,0.407042,0.530578,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
96654,-0.518324,1.971857,0.046747,410000,0,0.222758,1.519961,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
184712,-1.373395,-0.115681,0.701006,470000,0,1.081309,-1.080413,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129417,1.191818,-1.150755,1.156859,345000,0,0.025168,0.668312,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
111816,-1.373395,0.415589,-0.860030,650000,0,0.944925,-0.900775,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
142558,1.191818,-1.026917,-0.252331,565510,1,0.971536,-0.707245,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
160937,1.191818,-0.133720,-1.362129,250000,0,-0.114542,-0.200411,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# train.to_csv('train.csv', encoding='utf-8', index=False)
# test.to_csv('test.csv', encoding='utf-8', index=False)
# val.to_csv('val.csv', encoding='utf-8', index=False)